In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns 

# set the graphs to show in the jupyter notebook
%matplotlib inline

import datetime
from collections import Counter

In [2]:
online = pd.read_csv('https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/07_Visualization/Online_Retail/Online_Retail.csv', encoding='latin1')
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [3]:
online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
online.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
online.shape

(541909, 8)

### Create a histogram with the 10 countries that have the most 'Quantity' ordered except UK

In [ ]:
# UK를 제외한 'Quantity' 주문량이 가장 많은 10개 국가 대상으로 히스토그램 생성
countries = online.groupby('Country').sum()
countries.head()

In [ ]:
countries = countries.sort_values(by='Quantity', ascending=False)[1:11]

In [ ]:
countries['Quantity'].plot(kind='bar')

plt.xlabel('Countries')
plt.ylabel('Quantity')
plt.title('10 Countries with most orders')

plt.show()

### Exclude negative Quantity entries

In [ ]:
online = online[online['Quantity'] > 0]
online.head()

### Create a scatterplot with the Quantity per UnitPrice by CustomID for the top 3 Countries (except UK)

In [ ]:
# UK를 제외한 상위 3개 국가(CustomID별 단위 가격당 수량)에 대해 산점도 생성
customers = online.groupby(['Countries', 'CustomerID']).sum()
customers.head()

In [ ]:
# delete negative price
customers = customers[customers['UnitPrice'] > 0]

In [ ]:
customers['Country'] = customers.index.get_level_index(1) # column Country?

In [ ]:
top_countries = ['Netherlands', 'EIRE', 'Germany']

customers = customers[customers['Country'].isin(top_countries)]

In [ ]:
# Graph
graph = sns.FaceGrid(customers, col='Contry')

graph.map(plt.scatter, 'Quantity', 'UnitPrice', alpha=1)

graph.add_legend() # 범례 추가 안 한 그래프랑 비교해보기!!!

- **get_level_values()**: Pandas의 MultiIndex에서 특정 레벨의 값을 추출할 때 사용하는 함수 (특정 레벨만 보고 싶을 때 사용)
##### 레벨과 열은 다른 개념이다. 레벨은 인덱스 안의 각 계층 (예) 레벨 0 → '도시' 

- **add_legend():** seaborn에서 그래프에 범례(legend)를 추가하는 함수

### Investigate why the previous results look so uniformative.

This section might seem a bit tedious to go through. But I've thought of it as some kind of a simulation of problems one might encounter when dealing with data and other people. Besides there is a prize at the end. (i.e Section 8.) 

In [ ]:
# 이전 결과가 왜 그렇게 정보가 부족해보이는지 조사 

### Look at the first line of code in Step 6. And try to figure out if it leads to any kind of problem.

Display the first few rows of that DataFrame.

### Think about what the piece of code does and display the dtype of UnitPrice.

### Pull data from online_rt for CustromIDs 12346.0 and 12347.0.

### Reinterpreting the initial problem. 
To reiterate the question that we were dealing with:
"Create a scatterplot with the Quantity per UnitPrice by CustomerID for the top 3 Countries"

The question is open to a set of different interpretations. We need to disambiguate.

We could do a single plot by looking at all the data from the top 3 countries. Or we could do one plot per country. To keep things consistent with the rest of the exercise, let's stick to the latter oprion. So that's settled.

But "top 3 countries" with respect to what? Two answers suggest themselves: Total sales volume (i.e. total quantity sold) or total sales (i.e. revenue). This exercise goes for sales volume, so let's stick to that.

Step 7.2.1 Find out the top 3 countries in terms of sales volume.

### Step 7.2.2

Now that we have the top 3 countries, we can focus on the rest of the problem:
"Quantity per UnitPrice by CustomerID".
We need to unpack that.

"by CustomerID" part is easy. That means we're going to be plotting one dot per CustomerID's on our plot. In other words, we're going to be grouping by CustomerID.

"Quantity per UnitPrice" is trickier. Here's what we know:
*One axis will represent a Quantity assigned to a given customer. This is easy; we can just plot the total Quantity for each customer.
*The other axis will represent a UnitPrice assigned to a given customer. Remember a single customer can have any number of orders with different prices, so summing up prices isn't quite helpful. Besides it's not quite clear what we mean when we say "unit price per customer"; it sounds like price of the customer! A reasonable alternative is that we assign each customer the average amount each has paid per item. So let's settle that question in that manner.

### Modify, seect and plot data.
Step 7.3.1 Add a column to online_rt called Revenue calculate the revenue (Quantity * UnitPrice) from each sale.
We will use this later to figure out an average price per customer.